In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

def walkdir(path='/kaggle/input'):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
os.listdir('/kaggle/input')

In [ ]:
import sys
package_path = '../input/efficientnetpytorchpackage/EfficientNet-PyTorch-master'
sys.path.append(package_path)
from efficientnet_pytorch import EfficientNet

In [ ]:
from fastai.vision import *
import cv2
import fastai


print(torch.__version__)
print(fastai.__version__)

# Function Definitions

In [ ]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [ ]:


def get_prob(arr):
    #print(arr.shape)
    
    probs=torch.zeros(arr.size(0)+1,dtype=arr.dtype)
    probs[4]=arr[3]
    probs[3]=(arr[2]-arr[3]).relu()
    probs[2]=(arr[1]-arr[2]).relu()
    probs[1]=(arr[0]-arr[1]).relu()
    probs[0]=1-arr[0]

    return probs

def get_pred(probs):
    #print(arr.shape)
    return probs.argmax(0)

In [ ]:
base_image_dir = os.path.join('..', 'input/aptos2019-blindness-detection/')
train_dir = os.path.join(base_image_dir,'train_images/')
df = pd.read_csv(os.path.join(base_image_dir, 'train.csv'))

In [ ]:
def get_prob_s1(arr):
    #convert stage 1 preds to full model for all 5 levels
    # counts are from training set to calc naive bayes
    probs=torch.zeros(arr.size(0)+3,dtype=arr.dtype)
    probs[4]=arr[1]*295/1857
    probs[3]=arr[1]*193/1857
    probs[2]=arr[1]*999/1857
    probs[1]=arr[1]*370/1857
    probs[0]=arr[0]
    
    return probs

In [ ]:
def clahe(img):
    
    clahe = cv2.createCLAHE(clipLimit=2 , tileGridSize=(8,8))
    img_new_1 = clahe.apply(img[:,:,0])
    img_new_2 = clahe.apply(img[:,:,1])
    img_new_3 = clahe.apply(img[:,:,2])
    img_merge = cv2.merge([img_new_1,img_new_2,img_new_3])
    
    return img_merge

def clahel(img):
    
    image = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    clahe = cv2.createCLAHE(clipLimit=2 , tileGridSize=(8,8))
    img_new_1 = clahe.apply(image[:,:,0])
    img_merge = cv2.merge([img_new_1,image[:,:,1],image[:,:,2]])
    
    return cv2.cvtColor(img_merge,cv2.COLOR_LAB2RGB)


In [ ]:
def img_load(p,IMG_SIZE=512):
    image = cv2.imread(p)
    image = crop_image_from_gray(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width = image.shape[:2]
    m=max(height,width)
    
    f=IMG_SIZE/m
    image = cv2.resize(image, None,fx=f, fy=f)
    
    return image


def img_proc(image,blur_rad=10,use_clahel=False,use_clahe=False):
    
    if(use_clahel==True):
        image = clahel(image)
    if(use_clahe==True):
        image = clahe(image)
    
    if (blur_rad>0):
        image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0), blur_rad) ,-4 ,128)
    return Image(pil2tensor(image, np.float32).div_(255))

# Load Models

In [ ]:
walkdir('/kaggle/input/aptos-models')


In [ ]:
model_path='/kaggle/input/aptos-models/'
#No Processing
learn1=load_learner(model_path,'finalresnet34-all_train_images_512.pkl')
learn2=load_learner(model_path,'finalefficientnet-b6-all_train_images_512.pkl')
learn3=load_learner(model_path,'final-rn50-nz-nip.pkl')
#learn4=load_learner(model_path,'final-rn34-nz-nip.pkl')


#Blur=10
learn_b10_1=load_learner(model_path,'final-rn34-nonzero.pkl')
learn_b10_2=load_learner(model_path,'finalefficientnet-b6-all_train_images_512_blur.pkl')
learn_b10_3=load_learner(model_path,'finalresnet34-all_train_images_512_blur.pkl')

#Clahe
learn_clahe_1=load_learner(model_path,'finalresnet34-all_train_images_512_clahe.pkl')
learn_clahe_2=load_learner(model_path,'finalefficientnet-b6-all_train_images_512_clahe.pkl')

#Clahel
#learn_clahel_1=load_learner(model_path,'final-rn34-nz-clahel.pkl')
#learn_clahel_2=load_learner(model_path,'final-rn50-nz-clahel.pkl')
learn_clahel_3=load_learner(model_path,'finalresnet34-all_train_images_512_clahel.pkl')
learn_clahel_4=load_learner(model_path,'finalefficientnet-b6-all_train_images_512_clahel.pkl')

#Blur=50
#learn_blur_50_1=load_learner(model_path,'finalresnet34-all_train_images_512_blur_50.pkl')
#learn_blur_50_2=load_learner(model_path,'finalefficientnet-b6-all_train_images_512_blur_50.pkl')



In [ ]:
walkdir('/kaggle/input/aptos_2_stage')

In [ ]:
model_path='/kaggle/input/aptos_2_stage/'
#Stage 2 No Processing
learn5=load_learner(model_path,'final-s2resnet34-all_train_images_512.pkl')
learn6=load_learner(model_path,'final-s2-efficientnet-b6-all_train_images_512.pkl')

#Stage 2 Blur=10
learn_b10_4=load_learner(model_path,'final-s2-resnet34-all_train_images_512_blur.pkl')


#Stage 2 Clahe
learn_clahe_3=load_learner(model_path,'final-s2-resnet34-all_train_images_512_clahe.pkl')
learn_clahe_4=load_learner(model_path,'final-s2-efficientnet-b6-all_train_images_512_clahe.pkl')

#Stage 2 Clahel
learn_clahel_5=load_learner(model_path,'final-s2-resnet34-all_train_images_512_clahel.pkl')

#stage 1 model
learn_s1=load_learner('/kaggle/input/aptos_2_stage/','final-s1resnet34-all_train_images_512_clahe.pkl')

# Predict Test Set

In [ ]:
sample_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
sample_df.head()

In [ ]:
IMG_SIZE=512


test_path='/kaggle/input/aptos2019-blindness-detection/test_images/'
for it in progress_bar(sample_df.index):
    fns=str(sample_df.id_code[it])+'.png'

    p=test_path + fns

    image = cv2.imread(p)
    image = crop_image_from_gray(image)
    height, width = image.shape[:2]    
    m=max(height,width)            
    f=IMG_SIZE/m
    image_no_proc = cv2.resize(image, None,fx=f, fy=f)
    image_no_proc = cv2.cvtColor(image_no_proc, cv2.COLOR_BGR2RGB)
    image_blur_10=cv2.addWeighted ( image_no_proc,4, cv2.GaussianBlur( image_no_proc , (0,0), 10) ,-4 ,128)
    image_clahe = clahe(image_no_proc)
    image_clahel = clahel(image_no_proc)
    #image_blur_50 = cv2.addWeighted ( image_no_proc,4, cv2.GaussianBlur( image_no_proc , (0,0), 50) ,-4 ,128)

    
    

    probs=0
    
    

    #no processing
    imagef=Image(pil2tensor(image_no_proc, np.float32).div_(255))  

    pred_class,pred_idx,outputs = learn1.predict(imagef)
    probs=probs+get_prob(outputs)

    pred_class,pred_idx,outputs = learn2.predict(imagef)
    probs=probs+get_prob(outputs)
    
    pred_class,pred_idx,outputs = learn3.predict(imagef)
    probs=probs+get_prob(outputs)
    
    #pred_class,pred_idx,outputs = learn4.predict(imagef)
    #probs=probs+get_prob(outputs)
    
    pred_class,pred_idx,outputs = learn5.predict(imagef)
    probs=probs+get_prob(outputs)
    
    pred_class,pred_idx,outputs = learn6.predict(imagef)
    probs=probs+get_prob(outputs)

    #blur=10
    imagef=Image(pil2tensor(image_blur_10, np.float32).div_(255))

    pred_class,pred_idx,outputs = learn_b10_1.predict(imagef)
    probs=probs+get_prob(outputs)

    pred_class,pred_idx,outputs = learn_b10_2.predict(imagef)
    probs=probs+get_prob(outputs)

    pred_class,pred_idx,outputs = learn_b10_3.predict(imagef)
    probs=probs+get_prob(outputs)

    pred_class,pred_idx,outputs = learn_b10_4.predict(imagef)
    probs=probs+get_prob(outputs)



    #clahe
    imagef=Image(pil2tensor(image_clahe, np.float32).div_(255))
    
    # stage 1
    pred_class,pred_idx,outputs = learn_s1.predict(imagef)
    probs=probs+get_prob_s1(outputs)
    
    pred_class,pred_idx,outputs = learn_clahe_1.predict(imagef)
    probs=probs+get_prob(outputs)

    pred_class,pred_idx,outputs = learn_clahe_2.predict(imagef)
    probs=probs+get_prob(outputs)
    
    pred_class,pred_idx,outputs = learn_clahe_3.predict(imagef)
    probs=probs+get_prob(outputs)
    
    pred_class,pred_idx,outputs = learn_clahe_4.predict(imagef)
    probs=probs+get_prob(outputs)


    #clahel
    imagef=Image(pil2tensor(image_clahel, np.float32).div_(255))
    #pred_class,pred_idx,outputs = learn_clahel_1.predict(imagef)
    #probs=probs+get_prob(outputs)

    #pred_class,pred_idx,outputs = learn_clahel_2.predict(imagef)
    #probs=probs+get_prob(outputs)
    
    pred_class,pred_idx,outputs = learn_clahel_3.predict(imagef)
    probs=probs+get_prob(outputs)
    
    pred_class,pred_idx,outputs = learn_clahel_4.predict(imagef)
    probs=probs+get_prob(outputs)
    
    pred_class,pred_idx,outputs = learn_clahel_5.predict(imagef)
    probs=probs+get_prob(outputs)

    #blur=50

    #imagef=Image(pil2tensor(image_blur_50, np.float32).div_(255))
    #pred_class,pred_idx,outputs = learn_blur_50_1.predict(imagef)
    #probs=probs+get_prob(outputs)

    #pred_class,pred_idx,outputs = learn_blur_50_2.predict(imagef)
    #probs=probs+get_prob(outputs)

    preds2=get_pred(probs).numpy()
    #print(pred_class, pred_idx,outputs[pred_idx])
    #sample_df.diagnosis[it]=preds2.numpy()
    
    
    sample_df.loc[it,'diagnosis']=preds2
    #print(sample_df.loc[it,'diagnosis'])
sample_df[0:20]

In [ ]:
sample_df.to_csv('submission.csv',index=False)